## 2 Some simple forecasting methods
***
Some forecasting methods are extremely simple and surprisingly effective. We will use four simple forecasting methods as benchmarks throughout this book. To illustrate them, we will use quarterly Australian clay brick production between 1970 and 2004.

In [ ]:
# Import the libraries that we are going to use for the analysis:
import pandas as pd
import polars as pl

from statsforecast import StatsForecast
from statsforecast.models import *
from utilsforecast.plotting import plot_series

import matplotlib.pyplot as plt

In [ ]:
# Create a dataframe from a csv file:
df = pl.read_csv(
    "../Assets/aus-production.csv", separator=";", infer_schema_length=1000
)
df = df.filter(
    pl.col("Quarter").is_between(pl.lit("1970 Q1"), pl.lit("2004 Q4"))
).with_columns(
    pl.date_range(
        start=pl.lit("1970-01-01"), end=pl.lit("2004-12-31"), interval="1Q"
    ).alias("ds"),
    pl.lit(1).alias("unique_id"),
)

bricks = df.select("unique_id", "ds", pl.col("Bricks").cast(pl.Int16).alias("y"))


## Mean method

Here, the forecasts of all future values are equal to the average (or “mean”) of the historical data. If we let the historical data be denoted by $y_{1},...,y_{T}$, then we can write the forecasts as

\begin{gather*} 
\hat{y}_{T+h|T}=\bar{y}=(y_{1}+⋯+y_{T})/T
\end{gather*}

The notation $\hat{y}_{T+h|T}$ is a short-hand for the estimate of $y_{T+h}$ based on the data $y_{1},...,y_{T}$





In [ ]:
# HistoricAverage's usage example:
from statsforecast.models import HistoricAverage

model = StatsForecast(
    models=[HistoricAverage()],
    freq="1q",
    n_jobs=-1,
)
model = model.fit(df=bricks)
y_hat = model.predict(h=20)

In [ ]:
fig = plot_series(bricks, forecasts_df=y_hat, engine="plotly")
fig.update_layout(
    title="Clay bricks production in Australia",
)

<p style="text-align: center;">
Figure 3: Mean (or average) forecasts applied to clay brick production in Australia.
</p>

## Naïve method

For naïve forecasts, we simply set all forecasts to be the value of the last observation. That is,

\begin{gather*} 
\hat{y}_{T+h|T}=y_{T}
\end{gather*}

This method works remarkably well for many economic and financial time series.

In [ ]:
# HistoricAverage's usage example:
model = StatsForecast(
    models=[Naive()],
    freq="1q",
    n_jobs=-1,
)
model = model.fit(df=bricks)
y_hat = model.predict(h=20)
fig = plot_series(bricks, forecasts_df=y_hat, engine="plotly")
fig.update_layout(
    title="Clay bricks production in Australia",
)

<p style="text-align: center;">
Figure 4: Naïve forecasts applied to clay brick production in Australia.
</p>

Because a naïve forecast is optimal when data follow a random walk, these are also called random walk forecasts and the random walk model can be used instead of `NAIVE`.

## Seasonal naïve method

A similar method is useful for highly seasonal data. In this case, we set each forecast to be equal to the last observed value from the same season (e.g., the same month of the previous year). Formally, the forecast for time  $T+h$ is written as

\begin{gather*} 
\hat{y}_{T+h|T}=y_{T+h-m(k+1)}
\end{gather*}

where $m=$ the seasonal period, and $k$ is the integer part of  $(h−1)/m$ (i.e., the number of complete years in the forecast period prior to time $T+h)$. This looks more complicated than it really is. For example, with monthly data, the forecast for all future February values is equal to the last observed February value. With quarterly data, the forecast of all future Q2 values is equal to the last observed Q2 value (where Q2 means the second quarter). Similar rules apply for other months and quarters, and for other seasonal periods.

In [ ]:
# HistoricAverage's usage example:
model = StatsForecast(
    models=[SeasonalNaive(season_length=4)],
    freq="1q",
    n_jobs=-1,
)
model = model.fit(df=bricks)
y_hat = model.predict(h=20)
fig = plot_series(bricks, forecasts_df=y_hat, engine="plotly")
fig.update_layout(
    title="Clay bricks production in Australia",
)

<p style="text-align: center;">
Figure 5: Seasonal naïve forecasts applied to clay brick production in Australia.
</p>

## Drift method

A variation on the naïve method is to allow the forecasts to increase or decrease over time, where the amount of change over time (called the drift) is set to be the average change seen in the historical data. Thus the forecast for time $T+h$ is given by

\begin{gather*} 
\hat{y}_{T+h|T}=y_{T}+\frac{h}{T-1}\sum_{t=2}^{T}(y_{t}-y_{t-1})=y_{T}+h\frac{y_{T}-y_{1}}{T-1}
\end{gather*}

This is equivalent to drawing a line between the first and last observations, and extrapolating it into the future.

In [ ]:
# HistoricAverage's usage example:
model = StatsForecast(
    models=[RandomWalkWithDrift()],
    freq="1q",
    n_jobs=-1,
)
model = model.fit(df=bricks)
y_hat = model.predict(h=20)
fig = plot_series(bricks, forecasts_df=y_hat, engine="plotly")
fig.update_layout(
    title="Clay bricks production in Australia",
)

<p style="text-align: center;">
Figure 6: Drift forecasts applied to clay brick production in Australia.
</p>

## Example: Australian quarterly beer production

Figure 7 shows the first three methods applied to Australian quarterly beer production from 1992 to 2006, with the forecasts compared against actual values in the next 3.5 years.

In [ ]:
beer = df.select("unique_id", "ds", pl.col("Beer").cast(pl.Int16).alias("y"))

In [ ]:
# HistoricAverage's usage example:
model = StatsForecast(
    models=[Naive(), SeasonalNaive(season_length=4), RandomWalkWithDrift()],
    freq="1q",
    n_jobs=-1,
)
model = model.fit(df=beer)
y_hat = model.predict(h=20)
fig = plot_series(beer, forecasts_df=y_hat, engine="plotly")
fig.update_layout(
    title="Beer production in Australia",
)

<p style="text-align: center;">
Figure 7: Forecasts of Australian quarterly beer production.
</p>

In this case, only the seasonal naïve forecasts are close to the observed values from 2007 onwards.

## Example: Google’s daily closing stock price

In Figure 8, the non-seasonal methods are applied to Google’s daily closing stock price in 2015, and used to forecast one month ahead. Because stock prices are not observed every day, we first set up a new time index based on the trading days rather than calendar days.

In [ ]:
# Create a dataframe from a csv file:
google_stock = pl.read_csv("../Assets/GOOGL.csv").with_columns(
    pl.lit(1).alias("unique_id")
)

google_stock.head()

In [ ]:
# Filter the year of interest:
google_2015 = (
    google_stock.filter(pl.col("Date").le("2015-12-31"))
    .with_columns(pl.col("Date").str.to_datetime("%Y-%m-%d").alias("ds"))
    .drop("Date")
)
google_2016 = google_stock.filter(pl.col("Date").gt("2015-12-31")).with_columns(
    pl.col("Date").str.to_datetime("%Y-%m-%d").alias("ds")
)

google_2015.head()

In [ ]:
# HistoricAverage's usage example:
model = StatsForecast(
    models=[
        HistoricAverage(),
        Naive(),
        SeasonalNaive(season_length=4),
        RandomWalkWithDrift(),
    ],
    freq="1d",
    n_jobs=-1,
)
model = model.fit(
    df=google_2015,
    time_col="ds",
    target_col="Close",
)
y_hat = model.predict(h=len(google_2016))
fig = plot_series(
    google_2015,
    forecasts_df=y_hat,
    engine="plotly",
    target_col="Close",
)
fig.update_layout(
    title="Beer production in Australia",
)

<p style="text-align: center;">
Figure 8: Forecasts based on Google’s daily closing stock price in 2015.
</p>

Sometimes one of these simple methods will be the best forecasting method available; but in many cases, these methods will serve as benchmarks rather than the method of choice. That is, any forecasting methods we develop will be compared to these simple methods to ensure that the new method is better than these simple alternatives. If not, the new method is not worth considering.